In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from ucimlrepo import fetch_ucirepo

# 导入数据
bank_marketing = fetch_ucirepo(id=222)
X = bank_marketing.data.features
y = bank_marketing.data.targets['y']

# 1. 处理缺失值（避免SettingWithCopyWarning）
X = X.copy()  # 创建副本以避免切片问题
X['job'] = X['job'].fillna('unknown')
X['education'] = X['education'].fillna('unknown')
X['contact'] = X['contact'].fillna('unknown')
X = X.drop(columns=['poutcome'])  # 删除缺失值过多的特征

# 2. 处理右偏连续变量
continuous_cols = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
for col in continuous_cols:
    if col in ['balance', 'duration', 'campaign', 'pdays', 'previous']:  # 针对右偏特征
        if (X[col] <= 0).any():
            print(f"{col} contains non-positive values, applying log transformation with shift.")
            X[col] = np.log1p(X[col] - X[col].min() + 1)  # 移位确保正值
        else:
            X[col] = np.log1p(X[col])  # log(x+1)避免零值问题

# 3. 识别数值和类别特征
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# 4. 创建预处理管道（用于FCM、谱聚类、自编码器）
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols)
    ])

# 5. 保存原始数据（用于K-Prototypes）
X_raw = X.copy()

# 6. 应用预处理（生成独热编码数据）
X_processed = preprocessor.fit_transform(X)

# 7. 保存特征名称
feature_names = (numerical_cols.tolist() + 
                 preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols).tolist())

# 8. 验证数据
print("Processed data shape:", X_processed.shape)
print("Sample of processed data:\n", X_processed[:5])
print("Feature names:", feature_names)

# 9. 保存处理后的数据
X_processed_df = pd.DataFrame(X_processed, columns=feature_names)
X_processed_df.to_csv('processed_bank_data.csv', index=False)
X_raw.to_csv('raw_bank_data.csv', index=False)

balance contains non-positive values, applying log transformation with shift.
duration contains non-positive values, applying log transformation with shift.
pdays contains non-positive values, applying log transformation with shift.
previous contains non-positive values, applying log transformation with shift.
Processed data shape: (45211, 39)
Sample of processed data:
 [[ 1.60696496  0.50784905 -1.29847633  0.42939656 -0.90945712 -0.46629571
  -0.40589769  0.          0.          0.          1.          0.
   0.          0.          0.          0.          0.          0.
   1.          0.          0.          1.          0.          0.
   1.          0.          0.          1.          0.          0.
   0.          0.          0.          0.          0.          1.
   0.          0.          0.        ]
 [ 0.28852927 -0.59038661 -1.29847633 -0.16545896 -0.90945712 -0.46629571
  -0.40589769  0.          0.          0.          0.          0.
   0.          0.          0.          1.   

修正说明

解决Pandas警告：
使用X = X.copy()创建副本，避免切片问题。
替换X['job'].fillna('unknown', inplace=True)为X['job'] = X['job'].fillna('unknown')，符合Pandas 3.0的推荐做法。
移除可视化：
删除所有plt.figure、sns.histplot和plt.savefig相关代码，避免FileNotFoundError。
右偏处理：
保留对balance、duration、campaign、pdays、previous的对数转换，处理非正值（移位+np.log1p）。
age和day仅标准化，未应用对数转换（通常无强右偏）。
混合数据：
X_raw：保留原始类别变量，数值变量经对数转换和标准化，供K-Prototypes使用。
X_processed：数值变量标准化，类别变量独热编码，供FCM、谱聚类、自编码器使用。
验证与保存：
输出数据形状和样本，确保无异常。
保存processed_bank_data.csv和raw_bank_data.csv。
适配不平衡数据

聚类不直接使用y，但预处理确保右偏特征更均匀，减少对聚类结果的干扰。
后期可结合y（不平衡，yes较少）分析簇的订阅率。

In [6]:
from kmodes.kprototypes import KPrototypes
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load raw data
X_raw = pd.read_csv('raw_bank_data.csv')

# Define numerical and categorical columns (corrected to match feature names)
numerical_cols = ['age', 'balance', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous']
categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month']
categorical_indices = [X_raw.columns.get_loc(col) for col in categorical_cols]

# Standardize numerical columns
scaler = StandardScaler()
X_raw[numerical_cols] = scaler.fit_transform(X_raw[numerical_cols])

# K-Prototypes clustering
kproto = KPrototypes(n_clusters=4, init='Cao', n_init=10, random_state=42, verbose=1)
clusters = kproto.fit_predict(X_raw, categorical=categorical_indices)

# Add cluster labels
X_raw['Cluster'] = clusters

# Analyze clusters
cluster_summary = X_raw.groupby('Cluster').agg({
    **{col: 'mean' for col in numerical_cols},
    **{col: lambda x: x.mode()[0] for col in categorical_cols}
}).round(2)
print("Cluster Summary:\n", cluster_summary)

# Save results
X_raw.to_csv('kprototypes_clustered_data.csv', index=False)

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 15444, ncost: 312680.35543069267
Run: 1, iteration: 2/100, moves: 13041, ncost: 268746.8524046269
Run: 1, iteration: 3/100, moves: 5088, ncost: 264475.1562659198
Run: 1, iteration: 4/100, moves: 2255, ncost: 263713.8267412335
Run: 1, iteration: 5/100, moves: 2084, ncost: 263019.14932830294
Run: 1, iteration: 6/100, moves: 2921, ncost: 260902.65942490328
Run: 1, iteration: 7/100, moves: 2582, ncost: 259943.47685906643
Run: 1, iteration: 8/100, moves: 1378, ncost: 259667.26472747597
Run: 1, iteration: 9/100, moves: 725, ncost: 259594.79826940884
Run: 1, iteration: 10/100, moves: 435, ncost: 259569.67095147946
Run: 1, iteration: 11/100, moves: 221, ncost: 259563.59556565175
Run: 1, iteration: 12/100, moves: 136, ncost: 259561.06698478787
Run: 1, iteration: 13/100, moves: 115, ncost: 259559.19387445913
R

感谢您提供 K-Prototypes 聚类的结果！以下是对您提供的 `Cluster Summary` 的详细分析，结合银行营销数据集的背景（目标是预测客户是否订阅定期存款，数据包含数值型和类别型特征，目标变量 `y` 不平衡）。我将从业务角度解读每个簇的特征，分析其潜在客户画像、营销潜力，并提出针对性的营销策略建议。同时，我会考虑数据集中右偏特征（如 `balance`、`duration`）和目标变量的不平衡性（`yes` 较少）对结果的可能影响。

---

### 聚类结果概览

`Cluster Summary` 显示了 4 个簇（Cluster 0-3），每个簇的特征包括：
- **数值特征**（标准化后的均值）：`age`, `balance`, `day_of_week`, `duration`, `campaign`, `pdays`, `previous`。
- **类别特征**（众数）：`job`, `marital`, `education`, `default`, `housing`, `loan`, `contact`, `month`。

由于数值特征已标准化（均值 0，标准差 1），正值表示高于平均水平，负值表示低于平均水平。类别特征的众数反映该簇中最常见的类别。

---

### 逐簇分析

#### Cluster 0: “年轻蓝领，贷款负担型”
- **数值特征**：
  - `age: -0.14`：年龄略低于平均，偏年轻。
  - `balance: -0.27`：账户余额低于平均，可能经济压力较大。
  - `day_of_week: -0.29`：联系时间偏早（如周初）。
  - `duration: 0.23`：通话时长略高于平均，表明一定兴趣。
  - `campaign: -0.40`：联系次数低于平均，营销投入较少。
  - `pdays: -0.46`：较少被之前活动联系（接近 -1，可能未被联系）。
  - `previous: -0.40`：之前活动联系次数少。
- **类别特征**：
  - `job: blue-collar`：蓝领工人。
  - `marital: married`：已婚。
  - `education: secondary`：中学教育。
  - `default: no`：无违约记录。
  - `housing: yes`：有房贷。
  - `loan: no`：无个人贷款。
  - `contact: cellular`：通过手机联系。
  - `month: may`：5 月联系最多。
- **客户画像**：
  - 年轻（约 30-40 岁）、已婚的蓝领工人，教育水平中等，经济状况一般（低余额、有房贷）。
  - 对营销活动有一定响应（较长通话时长），但未被频繁联系。
- **营销潜力**：
  - **潜力中等**：较长的 `duration` 表明对定期存款有兴趣，但低余额和房贷可能限制购买力。
  - **风险**：经济压力可能降低订阅意愿。
- **营销策略**：
  - **产品**：推低门槛的定期存款产品（如小额存款计划）或与房贷相关的金融优惠。
  - **方式**：继续通过手机联系，强调产品安全性，安排在周初（如周一）联系。
  - **频率**：适度增加联系次数（当前 `campaign` 较低），但避免过度打扰。

#### Cluster 1: “中年管理层，低响应型”
- **数值特征**：
  - `age: 0.15`：年龄略高于平均，偏中年。
  - `balance: -0.24`：余额低于平均，经济状况一般。
  - `day_of_week: 0.73`：联系时间偏晚（如周四/周五）。
  - `duration: -0.54`：通话时长远低于平均，兴趣低。
  - `campaign: 1.08`：联系次数远高于平均，营销投入高。
  - `pdays: -0.46`：较少被之前活动联系。
  - `previous: -0.40`：之前活动联系少。
- **类别特征**：
  - `job: management`：管理人员。
  - `marital: married`：已婚。
  - `education: secondary`：中学教育。
  - `default: no`：无违约记录。
  - `housing: no`：无房贷。
  - `loan: no`：无个人贷款。
  - `contact: cellular`：手机联系。
  - `month: jul`：7 月联系最多。
- **客户画像**：
  - 中年（约 40-50 岁）、已婚的管理人员，教育中等，无贷款负担，但余额较低。
  - 对营销活动响应差（短通话时长），却被频繁联系（高 `campaign`）。
- **营销潜力**：
  - **潜力低**：低 `duration` 和高 `campaign` 表明客户兴趣低，可能是“硬性推销”目标，转化率可能低。
  - **风险**：继续高频联系可能引起反感。
- **营销策略**：
  - **产品**：暂停推定期存款，尝试低风险投资产品或储蓄计划，强调收益。
  - **方式**：减少电话联系频率，尝试数字化渠道（如邮件），在月末（如周五）联系。
  - **频率**：显著降低 `campaign` 次数，优化资源分配。

#### Cluster 2: “多次接触的高潜力型”
- **数值特征**：
  - `age: -0.01`：年龄接近平均（约 40 岁）。
  - `balance: -0.00`：余额接近平均，经济状况中等。
  - `day_of_week: -0.19`：联系时间略偏早。
  - `duration: 0.05`：通话时长略高于平均，兴趣一般。
  - `campaign: -0.26`：联系次数略低于平均。
  - `pdays: 2.11`：之前活动联系过，且间隔较短（高 `pdays`）。
  - `previous: 1.84`：之前活动联系次数多。
- **类别特征**：
  - `job: management`：管理人员。
  - `marital: married`：已婚。
  - `education: secondary`：中学教育。
  - `default: no`：无违约记录。
  - `housing: yes`：有房贷。
  - `loan: no`：无个人贷款。
  - `contact: cellular`：手机联系。
  - `month: may`：5 月联系最多。
- **客户画像**：
  - 中年（约 40 岁）、已婚的管理人员，教育中等，有房贷，经济状况稳定。
  - 曾被多次联系（高 `previous` 和 `pdays`），对当前活动有一定兴趣（略高 `duration`）。
- **营销潜力**：
  - **潜力高**：高 `previous` 和短间隔 `pdays` 表明客户对营销熟悉，略高的 `duration` 显示兴趣，转化潜力大。
  - **风险**：需避免过度营销导致疲劳。
- **营销策略**：
  - **产品**：推中高额的定期存款产品，结合房贷优惠（如存款抵扣贷款利息）。
  - **方式**：通过手机联系，强调产品回报率，安排在周初（如周二）。
  - **频率**：保持适度联系（当前 `campaign` 合理），可结合之前活动数据个性化沟通。

#### Cluster 3: “高余额精英型”
- **数值特征**：
  - `age: 0.46`：年龄明显高于平均，偏老年（约 50 岁或以上）。
  - `balance: 2.45`：余额远高于平均，经济实力强。
  - `day_of_week: 0.03`：联系时间平均（周中）。
  - `duration: 0.10`：通话时长略高于平均，兴趣一般。
  - `campaign: -0.12`：联系次数接近平均。
  - `pdays: -0.32`：较少被之前活动联系。
  - `previous: -0.30`：之前活动联系少。
- **类别特征**：
  - `job: management`：管理人员。
  - `marital: married`：已婚。
  - `education: tertiary`：高等教育。
  - `default: no`：无违约记录。
  - `housing: no`：无房贷。
  - `loan: no`：无个人贷款。
  - `contact: cellular`：手机联系。
  - `month: nov`：11 月联系最多。
- **客户画像**：
  - 较年长（约 50-60 岁）、已婚的管理人员，高学历，无贷款负担，账户余额极高。
  - 对营销活动有一定兴趣（略高 `duration`），但未被频繁联系。
- **营销潜力**：
  - **潜力最高**：高 `balance` 和 `tertiary` 教育表明强购买力，略高的 `duration` 显示兴趣。
  - **风险**：需提供高端产品以匹配其经济水平。
- **营销策略**：
  - **产品**：推高端定期存款或理财产品（如高收益债券），强调资产增值。
  - **方式**：通过手机联系，提供个性化咨询（如专属客户经理），安排在周中（如周三）。
  - **频率**：适度增加联系次数，重点在于高质量沟通而非高频推销。

---

### 综合分析与业务洞察

1. **客户分群概况**：
   - **Cluster 0**：年轻、负担重的蓝领，需低门槛产品。
   - **Cluster 1**：中年、低响应的管理层，需减少无效推销。
   - **Cluster 2**：多次接触的中年管理层，高潜力，需个性化策略。
   - **Cluster 3**：高余额、高学历的精英，优先目标。

2. **目标变量不平衡性（`y`）的影响**：
   - 数据中 `yes`（订阅定期存款）比例较低，需结合 `y` 分析各簇的订阅率：
     - **Cluster 3**（高余额、高学历）可能有较高订阅率，因其经济实力强。
     - **Cluster 2**（多次接触）也可能较高，因其对营销熟悉。
     - **Cluster 1**（低响应）可能订阅率最低，需优化资源分配。
     - **Cluster 0**（蓝领）订阅率可能中等，需低成本产品吸引。
   - 建议：加载 `y` 数据，计算各簇的订阅比例：
     ```python
     y = bank_marketing.data.targets['y'].map({'yes': 1, 'no': 0})
     X_raw['y'] = y
     subscription_rate = X_raw.groupby('Cluster')['y'].mean()
     print("Subscription Rate per Cluster:\n", subscription_rate)
     ```

3. **右偏特征的影响**：
   - 数值特征（如 `balance`、`duration`）已通过预处理（对数转换和标准化）缓解右偏影响，确保聚类结果不过度受极端值驱动。
   - `Cluster 3` 的高 `balance`（2.45）表明其捕捉了高净值客户，验证了预处理的有效性。

4. **营销资源分配**：
   - **优先级**：Cluster 3 > Cluster 2 > Cluster 0 > Cluster 1。
   - **资源优化**：减少对 Cluster 1 的高频联系（`campaign` 高但效果差），将资源转移到 Cluster 3 和 Cluster 2。

5. **渠道与时间优化**：
   - 所有簇主要通过 `cellular` 联系，建议继续优先使用手机。
   - 时间调整：
     - Cluster 0, 2：周初（如周一/二）。
     - Cluster 1：月末（如周五）。
     - Cluster 3：周中（如周三）。
   - 月份：Cluster 3（11 月）可能因年底财务规划更易响应，Cluster 0, 2（5 月）可能与春季活动相关。

---

### 营销策略总结

| **簇** | **客户画像**                     | **潜力** | **产品建议**                          | **联系方式**             | **联系时间**       | **频率**           |
|--------|----------------------------------|----------|---------------------------------------|--------------------------|--------------------|--------------------|
| 0      | 年轻蓝领，有房贷                 | 中等     | 小额存款计划，房贷优惠                | 手机                     | 周初（周一/二）    | 适度增加           |
| 1      | 中年管理层，低响应               | 低       | 低风险投资产品，储蓄计划              | 邮件，减少电话           | 月末（周五）       | 显著减少           |
| 2      | 中年管理层，多次接触             | 高       | 中高额存款，房贷抵扣                  | 手机                     | 周初（周二）       | 保持适度           |
| 3      | 高余额、高学历精英               | 最高     | 高端存款，理财产品                    | 手机，专属经理           | 周中（周三）       | 适度增加，高质量   |

---

### 进一步建议

1. **验证订阅率**：
   - 使用上述代码计算各簇的 `y` 均值，确认哪些簇是高潜力群体。
   - 若 Cluster 3 和 2 订阅率高，可进一步细分（如再运行 K-Prototypes，设置 `k=2` 针对这些簇）。

2. **优化高基数变量**：
   - `job`（12 类）和 `month`（12 类）可能影响聚类，可合并低频 `job`（如之前建议）：
     ```python
     job_counts = X_raw['job'].value_counts()
     low_freq_jobs = job_counts[job_counts < 100].index
     X_raw['job'] = X_raw['job'].replace(low_freq_jobs, 'other')
     ```

3. **补充轮廓系数**：
   - 肘部图（之前代码）可能无明显拐点，建议计算轮廓系数验证 `k=4` 的质量：
     ```python
     from sklearn.metrics import silhouette_score
     X_processed = pd.get_dummies(X_raw[categorical_cols], drop_first=True)
     X_processed[numerical_cols] = X_raw[numerical_cols]
     score = silhouette_score(X_processed, clusters)
     print(f"Silhouette Score for k=4: {score:.4f}")
     ```

4. **非凸簇考量**：
   - 若怀疑客户群体有复杂形状（如高余额但低订阅率的异常群体），可尝试谱聚类（Module 4）或 DBSCAN。

---

### 结论

- **Cluster 3** 是最有价值的客户群体（高余额、高学历），应优先投入资源。
- **Cluster 2** 具有高潜力，需个性化沟通。
- **Cluster 0** 可通过低门槛产品吸引，但需注意经济限制。
- **Cluster 1** 响应率低，需减少无效推销。
- 右偏特征和不平衡性的处理（预处理中的对数转换和聚类后结合 `y`）增强了结果的可靠性。

如果您希望进一步分析（如计算订阅率、调整 `k`、可视化簇分布），或需要其他方法的对比，请提供具体需求，我将提供相应代码和解读！

In [7]:
from kmodes.kprototypes import KPrototypes
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import numpy as np

# Load raw data
X_raw = pd.read_csv('raw_bank_data.csv')

# Handle high-cardinality categorical variables (e.g., merge low-frequency jobs)
job_counts = X_raw['job'].value_counts()
low_freq_jobs = job_counts[job_counts < 100].index
X_raw['job'] = X_raw['job'].replace(low_freq_jobs, 'other')

# Define numerical and categorical columns
numerical_cols = ['age', 'balance', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous']
categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month']
categorical_indices = [X_raw.columns.get_loc(col) for col in categorical_cols]

# Standardize numerical columns
scaler = StandardScaler()
X_raw[numerical_cols] = scaler.fit_transform(X_raw[numerical_cols])

# Select best k using silhouette score
silhouette_scores = []
k_range = range(3, 7)
for k in k_range:
    kproto = KPrototypes(n_clusters=k, init='Cao', n_init=5, gamma=0.5, random_state=42)
    clusters = kproto.fit_predict(X_raw, categorical=categorical_indices)
    # Compute silhouette score using processed data
    X_processed = pd.get_dummies(X_raw[categorical_cols], drop_first=True)
    X_processed[numerical_cols] = X_raw[numerical_cols]
    score = silhouette_score(X_processed, clusters)
    silhouette_scores.append(score)
    print(f"Silhouette Score for k={k}: {score:.4f}")

best_k = k_range[np.argmax(silhouette_scores)]
print(f"Best k: {best_k}")

# Run K-Prototypes with best k
kproto = KPrototypes(n_clusters=best_k, init='Cao', n_init=20, gamma=0.5, random_state=42, verbose=1)
clusters = kproto.fit_predict(X_raw, categorical=categorical_indices)

# Add cluster labels
X_raw['Cluster'] = clusters

# Analyze clusters
cluster_summary = X_raw.groupby('Cluster').agg({
    **{col: 'mean' for col in numerical_cols},
    **{col: lambda x: x.mode()[0] for col in categorical_cols}
}).round(2)
print("Cluster Summary:\n", cluster_summary)

# Save results
X_raw.to_csv('kprototypes_clustered_data.csv', index=False)

Silhouette Score for k=3: 0.1325
Silhouette Score for k=4: 0.1223
Silhouette Score for k=5: 0.1131
Silhouette Score for k=6: 0.1097
Best k: 3
Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 11145, ncost: 339134.0929727482
Run: 1, iteration: 2/100, moves: 9128, ncost: 323953.4488709607
Run: 1, iteration: 3/100, moves: 10830, ncost: 293521.28119781957
Run: 1, iteration: 4/100, moves: 5302, ncost: 281710.7990218432
Run: 1, iteration: 5/100, moves: 941, ncost: 281600.7691214053
Run: 1, iteration: 6/100, moves: 491, ncost: 281548.4948141563
Run: 1, iteration: 7/100, moves: 397, ncost: 281514.7785470409
Run: 1, iteration: 8/100, moves: 299, ncost: 281495.19017880276
Run: 1, iteration: 9/100, moves: 185, ncost: 281486.18321422697
Run: 1, iteration: 10/100, moves: 171, ncost: 281478.5767517988
Run: 1, iteration: 11/100, moves: 156, ncost: 281472.750394691